In [12]:
import json

import pandas as pd

character_metadata_path = "../../data/character.metadata.tsv"
movie_metadata_path = "../../data/movie.metadata.tsv"
name_cluster_path = "../../data/name.clusters.tsv"
plot_summaries_path = "../../data/plot_summaries.tsv"
tvtropes_path = "../../data/tvtropes.clusters.tsv"


characterMetadata = pd.read_csv(character_metadata_path, sep="\t")
movieMetadata = pd.read_csv(movie_metadata_path, sep="\t")
nameCluster = pd.read_csv(
    name_cluster_path,
    sep="\t",
    names=["Character name", "Freebase character/actor map ID"],
)
plotSummaries = pd.read_csv(
    plot_summaries_path, sep="\t", names=["Wikipedia movie ID", "plot"]
)
tvtropes = pd.read_csv(tvtropes_path, sep="\t", names=["trope", "details"])
tvtropes = pd.concat(
    [tvtropes["trope"], tvtropes["details"].apply(json.loads).apply(pd.Series)], axis=1
)

In [ ]:
print("Character Metadata")
display(characterMetadata.head())
print("Movie Metadata")
display(movieMetadata.head())
print("Name Cluster")
display(nameCluster.head())
print("Plot Summaries")
display(plotSummaries.head())
print("TV Tropes")
display(tvtropes.head())

In [14]:
movies = pd.merge(
    movieMetadata,
    characterMetadata,
    on=["Wikipedia movie ID", "Freebase movie ID"],
    how="inner",
)
movies = pd.merge(movies, plotSummaries, on="Wikipedia movie ID", how="inner")
movies = pd.merge(
    movies, nameCluster, on="Freebase character/actor map ID", how="inner"
)
# merge with tvtropes

In [ ]:
display(movies.head())